# 良/恶性乳腺癌肿瘤预测
### 良性/恶性乳腺癌肿瘤数据预处理

In [12]:
# 导入 numpy 与 pandas 工具包
import numpy as np
import pandas as pd

# 创建特征列表
column_names = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
               'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli',
               'Mitoses', 'Class']

# 使用 pandas.read_csv 从互联网上读取指定数据
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/'
                   +'breast-cancer-wisconsin.data', names=column_names)

# 将？替换为标准缺失值表示
data = data.replace(to_replace='?', value=np.nan)
# 丢弃带有缺失值的数据（只要有一个维度有缺失）
data = data.dropna(how='any')
# 输出 data 的数据量和维度
data.shape

(683, 11)

### 训练、测试数据

In [9]:
# 使用 sklearn.cross_validation 里的 train_test_split 模块用于分割数据。
from sklearn.cross_validation import train_test_split
# 随机采样25%数据用于测试，剩下的75%用于构建训练集
X_train, X_test, y_train, y_test = train_test_split(data[column_names[1:10]], data[column_names[10]], test_size=0.25, 
                                                    random_state=33)

In [10]:
# 查验训练样本的数量和类别分布
y_train.value_counts()

2    344
4    168
Name: Class, dtype: int64

In [11]:
# 查验测试样本的数量和类别分布。
y_test.value_counts()

2    100
4     71
Name: Class, dtype: int64

### 使用线性分类模型从事预测任务

In [27]:
# 从sklearn.preprocessing里导入 StandardScaler。
from sklearn.preprocessing import StandardScaler
# 从sklearn.linear_model 里导入LogisticRegression 与 SGDClassifier。
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# 标准化数据，保证每个维度的特征数据方差为1，均值为0.使得预测结果不会被某些维度过大的特征值而主导。
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

# 初始化 LogisticRegression 与 SGDClassifier
lr = LogisticRegression()
sgdc = SGDClassifier()
# 调用LogisticRegression中的fit函数/模块用来训练模型参数
lr.fit(X_train, y_train)
# 使用训练好的模型lr对X_test进行预测，结果存储在变量 lr_y_predict 中
lr_y_predict = lr.predict(X_test)
# 调用 SGDClassifier 中的fit函数/模块用来训练模型参数
sgdc.fit(X_train, y_train)
# 使用训练好的模型 sgdc 对 X_test 进行预测，结果存储在变量 sgdc_y_predict 中
sgdc_y_predict = sgdc.predict(X_test)

/Users/zhaoyulai/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### 性能分析

In [28]:
# 从sklearn.metrics 中导入classification_report模块
from sklearn.metrics import classification_report

# 使用逻辑斯蒂回归模型自带的评分函数 score 获得模型在测试集上的准确性结果
print('Accuracy of LR Classifier:', lr.score(X_test,y_test))
# 利用classification_report模块获得LogisticRegression其他三个指标的结果
print(classification_report(y_test, lr_y_predict, target_names=['Benign', 'Malignant']))

Accuracy of LR Classifier: 0.9883040935672515
             precision    recall  f1-score   support

     Benign       0.99      0.99      0.99       100
  Malignant       0.99      0.99      0.99        71

avg / total       0.99      0.99      0.99       171



In [30]:
# 使用随机梯度下降模型自带的评分函数 score 获得模型在测试集上的准确性结果
print('Accuracy of SGD Classifier:', sgdc.score(X_test,y_test))
# 利用classification_report模块获得 SGDClassifier 其他三个指标的结果
print(classification_report(y_test, sgdc_y_predict, target_names=['Benign', 'Malignant']))

Accuracy of SGD Classifier: 0.9824561403508771
             precision    recall  f1-score   support

     Benign       1.00      0.97      0.98       100
  Malignant       0.96      1.00      0.98        71

avg / total       0.98      0.98      0.98       171

